In [10]:
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import time
from io import StringIO
import random

import pandas_datareader.data as web
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas_profiling
import copy

sns.set()

## 取得財報 ##
* [公開資訊觀測站：單一公司案例文件查詢及下載](https://mops.twse.com.tw/mops/web/t203sb01)

In [109]:
def financial_statement(stock):

    now = datetime.now() #現在的時間
    year = now.strftime("%Y") #抓今年

    df_compsheet = pd.DataFrame()
    # df_compsheet.index.name = '會計項目Accounting Title'

    for y in range(int(year),2008,-1):
        ### 先與網站請求抓到每天的報價資料 ###
        print (y)
        if y > 2012:
            url = 'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=' + stock + '&SYEAR=%d&SSEASON=4&REPORT_ID=C' %(y)
            print (url)

            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                # 1: 資產負債表 
                # 2: 綜合損益表
                # 3: 現金流量表

                getdata[2].drop(getdata[2].head(1).index, inplace=True)
                getdata[2] = getdata[2].reset_index(drop=True)
                getdata[2].index = getdata[2].iloc[: , 0]
                getdata[2].drop(getdata[2].columns[0], axis=1, inplace=True)
                for col in getdata[2].columns:
                    getdata[2][col] = pd.to_numeric(getdata[2][col], 'coerce')

                getdata[2] = getdata[2].dropna()
                
                getdata[1].drop(getdata[1].head(1).index, inplace=True)
                getdata[1] = getdata[1].reset_index(drop=True)
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for (col1,col2) in zip(getdata[1].columns, getdata[2].columns):
                    getdata[1][col1] = pd.to_numeric(getdata[1][col1], 'coerce')
                    getdata[1].rename(columns={col1:col2}, inplace=True)

                getdata[1] = getdata[1].dropna()

                getdata[3].drop(getdata[3].head(1).index, inplace=True)
                getdata[3] = getdata[3].reset_index(drop=True)
                getdata[3].index = getdata[3].iloc[: , 0]
                getdata[3].drop(getdata[3].columns[0], axis=1, inplace=True)
                for col in getdata[3].columns:
                    getdata[3][col] = pd.to_numeric(getdata[3][col], 'coerce')

                getdata[3] = getdata[3].dropna()

                getdata[1] = getdata[1].T
                getdata[2] = getdata[2].T
                getdata[3] = getdata[3].T

                getdata[2] = getdata[2][['營業收入合計','營業毛利（毛損）','營業利益（損失）',
                                         '繼續營業單位稅前淨利（淨損）','營業費用合計','本期淨利（淨損）','基本每股盈餘合計']]
                getdata[2]['本期現金及約當現金增加（減少）數'] = getdata[3][['本期現金及約當現金增加（減少）數']]
                
                if y==2017 or y==2014 or y==2013:
                    getdata[2]['股東權益總額'] = getdata[1]['權益總額']
                else:
                    getdata[2]['股東權益總額'] = getdata[1]['權益總計']
                
                if y==2014 or y==2013:
                    getdata[2]['資產總額'] = getdata[1]['資產總額']
                else:
                    getdata[2]['資產總額'] = getdata[1]['資產總計']

                getdata[2] = getdata[2].T

                df_compsheet['%d'%(y+1)] = getdata[2][getdata[2].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')

        else:
            url = url = 'https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01Form&step=0&comp_id='+stock+'&YEAR1=%d&SEASON1=4&R_TYPE1=B' %(y)
            print (url)

            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                getdata[1] = getdata[1].dropna()
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for col in getdata[1].columns:
                    getdata[1][col] = pd.to_numeric(getdata[1][col], 'coerce')

                getdata[1].index.name = ''
                getdata[1] = getdata[1].T

                getdata[1] = getdata[1][['營業收入合計','營業毛利(毛損)','營業淨利(淨損)',
                                         '繼續營業單位稅前淨利(淨損)','營業費用合計','合併總損益','基本每股盈餘淨額',
                                         '本期現金及約當現金淨增減數','股東權益總額','資產總額']]

                col_names = ['營業收入合計','營業毛利（毛損）','營業利益（損失）',
                            '繼續營業單位稅前淨利（淨損）','營業費用合計','本期淨利（淨損）','基本每股盈餘合計','本期現金及約當現金增加（減少）數','股東權益總額','資產總額']
                for iname in range(len(col_names)):
                    getdata[1].rename(columns={getdata[1].columns[iname]: col_names[iname]}, inplace=True)

                getdata[1] = getdata[1].T

                df_compsheet['%d'%(y+1)] = getdata[1][getdata[1].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')

        #要睡覺一下，不然會被ben掉
        time.sleep(5)

    ### in case there is anything wrong
    ### skip the company -> check later
    try:
        df_compsheet.index.name = ''
        df_compsheet = df_compsheet.T
        df_compsheet.index = pd.to_datetime(df_compsheet.index)
        for col in df_compsheet.columns:
            df_compsheet[col] = pd.to_numeric(df_compsheet[col], 'coerce')

        df_compsheet['營業毛利率(%)'] = df_compsheet.apply(lambda row: row['營業毛利（毛損）'] / row['營業收入合計'] * 100., axis=1)
        df_compsheet['營業利益率(%)'] = df_compsheet.apply(lambda row: row['營業利益（損失）'] / row['營業收入合計'] * 100., axis=1)
        df_compsheet['純益率(%)'] = df_compsheet.apply(lambda row: row['繼續營業單位稅前淨利（淨損）'] / row['營業收入合計'] * 100., axis=1)
        df_compsheet['股東權益率(%)'] = df_compsheet.apply(lambda row: row['繼續營業單位稅前淨利（淨損）'] / row['股東權益總額'] * 100., axis=1)
        df_compsheet['總資產報酬率(%)'] = df_compsheet.apply(lambda row: row['繼續營業單位稅前淨利（淨損）'] / row['資產總額'] * 100., axis=1)

        df_compsheet = df_compsheet.sort_index()
        df_compsheet['營收成長率'] = df_compsheet['營業收入合計'].pct_change()
        df_compsheet['淨利成長率'] = df_compsheet['繼續營業單位稅前淨利（淨損）'].pct_change()
        
    except Exception as e:
        print('Errors have occured! Check company ' + stock)
        
    df_compsheet.to_csv('./Financial_statement/'+stock+'.csv', encoding='utf-8')
    
    return df_compsheet


In [110]:
with open('Monthly-revenue-201211to201911.pickle', 'rb') as file:
    data = pickle.load(file)
    
file.close()

In [ ]:
data_fstatement = {}

icomp = 1
for sym in data['2019-11-01']['公司代號']:
    if sym == '全部國內上市公司合計':
        continue
        
    print ('Process: %d/%d' % (icomp,len(data['2019-11-01']['公司代號'])),'company symbol: ',sym)
    data_fstatement[sym] = financial_statement(sym)
    icomp = icomp + 1



In [114]:
# save the dictionary to a file
file = open('Financial-statement-summary.pickle', 'wb')
pickle.dump(data_fstatement, file)
file.close()

### 合併財報：個別公司 ###

In [ ]:
df_compsheet = financial_statement('2534')


(df_compsheet.style
     .background_gradient('Reds',subset='營業收入合計')
     .background_gradient('Reds',subset='營業毛利（毛損）')
     .background_gradient('coolwarm',subset='營業利益（損失）')
     .background_gradient('coolwarm',subset='繼續營業單位稅前淨利（淨損）')
     .background_gradient('coolwarm',subset='基本每股盈餘合計')
     .background_gradient('coolwarm',subset='本期現金及約當現金增加（減少）數')
     .background_gradient('coolwarm',subset='營業毛利率(%)')
     .background_gradient('coolwarm',subset='營業利益率(%)')
     .background_gradient('coolwarm',subset='純益率(%)')
     .background_gradient('coolwarm',subset='股東權益率(%)')
     .background_gradient('coolwarm',subset='總資產報酬率(%)')
     .background_gradient('coolwarm',subset='營收成長率')
     .background_gradient('coolwarm',subset='淨利成長率')
     # .background_gradient('Greens', subset='Google search: Stock')
     .highlight_null()
)

## 取得財報：針對沒有提供合併財報的公司 ##

In [161]:
### Alternative function to fetch financial statement
### for those who do not have combined statement

def financial_statement_single(stock):
    now = datetime.now() #現在的時間
    year = now.strftime("%Y") #抓今年

    df_compsheet = pd.DataFrame()
    # df_compsheet.index.name = '會計項目Accounting Title'

    for y in range(int(year),2008,-1):
        print (y)
        
        if y > 2012:
            url = 'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID='+stock+'&SYEAR=%d&SSEASON=4&REPORT_ID=A' %(y)
            print (url)
        
            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                # 1: 資產負債表 
                # 2: 綜合損益表
                # 3: 現金流量表

                getdata[1].drop(getdata[1].head(1).index, inplace=True)
                getdata[1] = getdata[1].reset_index(drop=True)
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for col in getdata[1].columns:
                    getdata[1][col] = pd.to_numeric(getdata[1][col], 'coerce')

                getdata[1] = getdata[1].dropna()

                getdata[0].drop(getdata[0].head(1).index, inplace=True)
                getdata[0] = getdata[0].reset_index(drop=True)
                getdata[0].index = getdata[0].iloc[: , 0]
                getdata[0].drop(getdata[0].columns[0], axis=1, inplace=True)
                for (col1,col2) in zip(getdata[0].columns, getdata[1].columns):
                    getdata[0][col1] = pd.to_numeric(getdata[0][col1], 'coerce')
                    getdata[0].rename(columns={col1:col2}, inplace=True)

                getdata[1] = getdata[1].dropna()

                getdata[2].drop(getdata[2].head(1).index, inplace=True)
                getdata[2] = getdata[2].reset_index(drop=True)
                getdata[2].index = getdata[2].iloc[: , 0]
                getdata[2].drop(getdata[2].columns[0], axis=1, inplace=True)
                for col in getdata[2].columns:
                    getdata[2][col] = pd.to_numeric(getdata[2][col], 'coerce')

                getdata[2] = getdata[2].dropna()

                getdata[0] = getdata[0].T
                getdata[1] = getdata[1].T
                getdata[2] = getdata[2].T

                getdata[1] = getdata[1][['營業收入合計','營業毛利（毛損）','營業利益（損失）',
                                        '繼續營業單位稅前淨利（淨損）','營業費用合計','本期淨利（淨損）','基本每股盈餘合計']]
                getdata[1]['本期現金及約當現金增加（減少）數'] = getdata[2][['本期現金及約當現金增加（減少）數']]

                if y==2017:
                    getdata[1]['股東權益總額'] = getdata[0]['權益總計']
                else:
                    getdata[1]['股東權益總額'] = getdata[0]['權益總額']

                if y==2014 or y==2013:
                    getdata[1]['資產總額'] = getdata[0]['資產總額']
                else:
                    getdata[1]['資產總額'] = getdata[0]['資產總計']

                getdata[1] = getdata[1].T

                df_compsheet['%d'%(y+1)] = getdata[1][getdata[1].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')
                
        else:
            url = 'https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01Form&step=0&comp_id='+stock+'&YEAR1=%d&SEASON1=4&R_TYPE1=A' %(y)
            print (url)

            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                getdata[1] = getdata[1].dropna()
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for col in getdata[1].columns:
                    getdata[1][col] = pd.to_numeric(getdata[1][col], 'coerce')

                getdata[1].index.name = ''
                getdata[1] = getdata[1].T

                getdata[1] = getdata[1][['營業收入合計','營業毛利(毛損)','營業淨利(淨損)',
                                         '繼續營業單位稅前淨利(淨損)','營業費用合計','本期淨利(淨損)','基本每股盈餘淨額',
                                         '本期現金及約當現金淨增減數','股東權益總額','資產總額']]

                getdata[1] = getdata[1].loc[:,~getdata[1].columns.duplicated()]
                
                col_names = ['營業收入合計','營業毛利（毛損）','營業利益（損失）',
                            '繼續營業單位稅前淨利（淨損）','營業費用合計','本期淨利（淨損）','基本每股盈餘合計','本期現金及約當現金增加（減少）數','股東權益總額','資產總額']
                
                for iname in range(len(col_names)):
                    getdata[1].rename(columns={getdata[1].columns[iname]: col_names[iname]}, inplace=True)

                getdata[1] = getdata[1].T

                df_compsheet['%d'%(y+1)] = getdata[1][getdata[1].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')

        #要睡覺一下，不然會被ben掉
        time.sleep(5)
        
    ### in case there is anything wrong
    ### skip the company -> check later
    try:
        df_compsheet.index.name = ''
        df_compsheet = df_compsheet.T
        df_compsheet.index = pd.to_datetime(df_compsheet.index)
        for col in df_compsheet.columns:
            df_compsheet[col] = pd.to_numeric(df_compsheet[col], 'coerce')

        df_compsheet['營業毛利率(%)'] = df_compsheet.apply(lambda row: row['營業毛利（毛損）'] / row['營業收入合計'] * 100., axis=1)
        df_compsheet['營業利益率(%)'] = df_compsheet.apply(lambda row: row['營業利益（損失）'] / row['營業收入合計'] * 100., axis=1)
        df_compsheet['純益率(%)'] = df_compsheet.apply(lambda row: row['繼續營業單位稅前淨利（淨損）'] / row['營業收入合計'] * 100., axis=1)
        df_compsheet['股東權益率(%)'] = df_compsheet.apply(lambda row: row['繼續營業單位稅前淨利（淨損）'] / row['股東權益總額'] * 100., axis=1)
        df_compsheet['總資產報酬率(%)'] = df_compsheet.apply(lambda row: row['繼續營業單位稅前淨利（淨損）'] / row['資產總額'] * 100., axis=1)

        df_compsheet = df_compsheet.sort_index()
        df_compsheet['營收成長率'] = df_compsheet['營業收入合計'].pct_change()
        df_compsheet['淨利成長率'] = df_compsheet['繼續營業單位稅前淨利（淨損）'].pct_change()
        
    except Exception as e:
        print('Errors have occured! Check company ' + stock)
        
    df_compsheet.to_csv('./Financial_statement/'+stock+'.csv', encoding='utf-8')

    return df_compsheet

In [162]:
df_compsheet = financial_statement_single('6281')


(df_compsheet.style
     .background_gradient('Reds',subset='營業收入合計')
     .background_gradient('Reds',subset='營業毛利（毛損）')
     .background_gradient('coolwarm',subset='營業利益（損失）')
     .background_gradient('coolwarm',subset='繼續營業單位稅前淨利（淨損）')
     .background_gradient('coolwarm',subset='基本每股盈餘合計')
     .background_gradient('coolwarm',subset='本期現金及約當現金增加（減少）數')
     .background_gradient('coolwarm',subset='營業毛利率(%)')
     .background_gradient('coolwarm',subset='營業利益率(%)')
     .background_gradient('coolwarm',subset='純益率(%)')
     .background_gradient('coolwarm',subset='股東權益率(%)')
     .background_gradient('coolwarm',subset='總資產報酬率(%)')
     .background_gradient('coolwarm',subset='營收成長率')
     .background_gradient('coolwarm',subset='淨利成長率')
     # .background_gradient('Greens', subset='Google search: Stock')
     .highlight_null()
)

2019
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=6281&SYEAR=2019&SSEASON=4&REPORT_ID=A
get 404, please check if anything is wrong
2018
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=6281&SYEAR=2018&SSEASON=4&REPORT_ID=A
2017
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=6281&SYEAR=2017&SSEASON=4&REPORT_ID=A
2016
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=6281&SYEAR=2016&SSEASON=4&REPORT_ID=A
2015
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=6281&SYEAR=2015&SSEASON=4&REPORT_ID=A
2014
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=6281&SYEAR=2014&SSEASON=4&REPORT_ID=A
2013
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=6281&SYEAR=2013&SSEASON=4&REPORT_ID=A
2012
https://mops.twse.com.tw/server-java/t147sb02?t203sb01Fo

,營業收入合計,營業毛利（毛損）,營業利益（損失）,繼續營業單位稅前淨利（淨損）,營業費用合計,本期淨利（淨損）,基本每股盈餘合計,本期現金及約當現金增加（減少）數,股東權益總額,資產總額,營業毛利率(%),營業利益率(%),純益率(%),股東權益率(%),總資產報酬率(%),營收成長率,淨利成長率
2010-01-01 00:00:00,1.19278e+07,2.69072e+06,474960,514452,2.21576e+06,392203,2.97,508764,2.09023e+06,4.2567e+06,22.5583,3.98195,4.31304,24.6122,12.0857,nan,nan
2011-01-01 00:00:00,1.20251e+07,2.693e+06,495571,520251,2.19743e+06,431718,4.35,-261273,2.14822e+06,4.08049e+06,22.3948,4.12113,4.32637,24.2177,12.7497,0.00815715,0.0112722
2012-01-01 00:00:00,1.2679e+07,2.85576e+06,522814,550999,2.33295e+06,458405,4.62,120599,2.21172e+06,4.59651e+06,22.5235,4.12346,4.34575,24.9127,11.9873,0.0543782,0.0591022
2013-01-01 00:00:00,1.49805e+07,3.28088e+06,608699,649522,2.67218e+06,537593,5.42,-59716,2.29344e+06,4.95147e+06,21.901,4.06328,4.33579,28.3209,13.1178,0.181518,0.178808
2014-01-01 00:00:00,1.47309e+07,3.10924e+06,467079,498850,2.64217e+06,415827,4.19,17118,2.15526e+06,5.00883e+06,21.1069,3.17074,3.38642,23.1457,9.95941,-0.0166601,-0.231974
2015-01-01 00:00:00,1.59383e+07,3.29503e+06,476653,511342,2.81838e+06,426896,4.3,-59509,2.23796e+06,5.23715e+06,20.6737,2.99061,3.20826,22.8486,9.76375,0.0819629,0.0250416
2016-01-01 00:00:00,1.65017e+07,3.3395e+06,489954,522494,2.84955e+06,433782,4.37,572427,2.26431e+06,5.45179e+06,20.2373,2.96911,3.1663,23.0752,9.5839,0.0353502,0.0218093
2017-01-01 00:00:00,1.68124e+07,3.42493e+06,575828,595543,2.8491e+06,494069,4.98,-327996,2.37081e+06,5.08617e+06,20.3715,3.42502,3.54229,25.1198,11.7091,0.0188254,0.139808
2018-01-01 00:00:00,1.63727e+07,3.21185e+06,524505,545738,2.68735e+06,452916,4.57,-233071,2.38286e+06,4.80925e+06,19.6172,3.20354,3.33323,22.9027,11.3477,-0.0261546,-0.0836296
2019-01-01 00:00:00,1.63433e+07,3.14052e+06,469890,506910,2.67063e+06,405437,4.09,-87373,2.39993e+06,4.50912e+06,19.2159,2.87512,3.10164,21.1219,11.2419,-0.00179348,-0.0711477


## 取得財報：針對金融類公司 ##

In [187]:
def financial_statement_bank(stock):

    now = datetime.now() #現在的時間
    year = now.strftime("%Y") #抓今年

    df_compsheet = pd.DataFrame()
    # df_compsheet.index.name = '會計項目Accounting Title'

    for y in range(int(year),2008,-1):
        ### 先與網站請求抓到每天的報價資料 ###
        print (y)
        if y > 2012:
            url = 'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=' + stock + '&SYEAR=%d&SSEASON=4&REPORT_ID=C' %(y)
            print (url)

            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                # 1: 資產負債表 
                # 2: 綜合損益表
                # 3: 現金流量表

                getdata[2].drop(getdata[2].head(1).index, inplace=True)
                getdata[2] = getdata[2].reset_index(drop=True)
                getdata[2].index = getdata[2].iloc[: , 0]
                getdata[2].drop(getdata[2].columns[0], axis=1, inplace=True)
                for col in getdata[2].columns:
                    getdata[2][col] = pd.to_numeric(getdata[2][col], 'coerce')

                getdata[2] = getdata[2].dropna()
                
                getdata[1].drop(getdata[1].head(1).index, inplace=True)
                getdata[1] = getdata[1].reset_index(drop=True)
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for (col1,col2) in zip(getdata[1].columns, getdata[2].columns):
                    getdata[1][col1] = pd.to_numeric(getdata[1][col1], 'coerce')
                    getdata[1].rename(columns={col1:col2}, inplace=True)

                getdata[1] = getdata[1].dropna()

                getdata[3].drop(getdata[3].head(1).index, inplace=True)
                getdata[3] = getdata[3].reset_index(drop=True)
                getdata[3].index = getdata[3].iloc[: , 0]
                getdata[3].drop(getdata[3].columns[0], axis=1, inplace=True)
                for col in getdata[3].columns:
                    getdata[3][col] = pd.to_numeric(getdata[3][col], 'coerce')

                getdata[3] = getdata[3].dropna()

                getdata[1] = getdata[1].T
                getdata[2] = getdata[2].T
                getdata[3] = getdata[3].T

                getdata[2] = getdata[2][['繼續營業單位稅前淨利（淨損）','基本每股盈餘合計']]
                getdata[2]['本期現金及約當現金增加（減少）數'] = getdata[3][['本期現金及約當現金增加（減少）數']]

                getdata[2] = getdata[2].T

                df_compsheet['%d'%(y+1)] = getdata[2][getdata[2].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')

        else:
            url = url = 'https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01Form&step=0&comp_id='+stock+'&YEAR1=%d&SEASON1=4&R_TYPE1=B' %(y)
            print (url)

            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                getdata[1] = getdata[1].dropna()
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for col in getdata[1].columns:
                    getdata[1][col] = pd.to_numeric(getdata[1][col], 'coerce')

                getdata[1].index.name = ''
                getdata[1] = getdata[1].T

                getdata[1] = getdata[1][['繼續營業單位稅前淨利(淨損)','基本每股盈餘合計',
                                         '本期現金及約當現金淨增減數']]

                col_names = ['繼續營業單位稅前淨利（淨損）','基本每股盈餘合計','本期現金及約當現金增加（減少）數']
                for iname in range(len(col_names)):
                    getdata[1].rename(columns={getdata[1].columns[iname]: col_names[iname]}, inplace=True)

                getdata[1] = getdata[1].T

                df_compsheet['%d'%(y+1)] = getdata[1][getdata[1].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')

        #要睡覺一下，不然會被ben掉
        time.sleep(5)

    ### in case there is anything wrong
    ### skip the company -> check later
    try:
        df_compsheet.index.name = ''
        df_compsheet = df_compsheet.T
        df_compsheet.index = pd.to_datetime(df_compsheet.index)
        for col in df_compsheet.columns:
            df_compsheet[col] = pd.to_numeric(df_compsheet[col], 'coerce')

        df_compsheet = df_compsheet.sort_index()
        df_compsheet['淨利成長率'] = df_compsheet['繼續營業單位稅前淨利（淨損）'].pct_change()
        
    except Exception as e:
        print('Errors have occured! Check company ' + stock)
        
    df_compsheet.to_csv('./Financial_statement/'+stock+'.csv', encoding='utf-8')
    
    return df_compsheet


In [204]:
df_compsheet = financial_statement_bank('6005')
df_compsheet

2019
https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2019&SSEASON=4&REPORT_ID=C
get 404, please check if anything is wrong
2018
https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2018&SSEASON=4&REPORT_ID=C
2017
https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2017&SSEASON=4&REPORT_ID=C
2016
https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2016&SSEASON=4&REPORT_ID=C
2015
https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2015&SSEASON=4&REPORT_ID=C
2014
https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2014&SSEASON=4&REPORT_ID=C
2013
https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=6005&SYEAR=2013&SSEASON=4&REPORT_ID=C
2012
https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01Form&step=0&comp_id=6005&YEAR1=2012&SEASON1=4&R_TYPE1=B
get 404, please check if anything is wrong
2011
https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01

,繼續營業單位稅前淨利（淨損）,基本每股盈餘合計,本期現金及約當現金增加（減少）數,淨利成長率
2014-01-01,1793877.0,0.62,-854169.0,NaN
2015-01-01,2432701.0,0.85,-326761.0,0.356114
2016-01-01,2034035.0,0.66,498915.0,-0.163878
2017-01-01,1672233.0,0.53,696917.0,-0.177874
2018-01-01,3503637.0,1.33,1680662.0,1.095185
2019-01-01,2326530.0,0.61,889342.0,-0.335967


## 取得財報：針對不提供合併財報的金融類公司 ##

In [191]:
### Alternative function to fetch financial statement
### for those who do not have combined statement

def financial_statement_bank_single(stock):
    now = datetime.now() #現在的時間
    year = now.strftime("%Y") #抓今年

    df_compsheet = pd.DataFrame()
    # df_compsheet.index.name = '會計項目Accounting Title'

    for y in range(int(year),2008,-1):
        print (y)
        
        if y > 2012:
            url = 'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID='+stock+'&SYEAR=%d&SSEASON=4&REPORT_ID=A' %(y)
            print (url)
        
            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                # 1: 資產負債表 
                # 2: 綜合損益表
                # 3: 現金流量表

                getdata[1].drop(getdata[1].head(1).index, inplace=True)
                getdata[1] = getdata[1].reset_index(drop=True)
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for col in getdata[1].columns:
                    getdata[1][col] = pd.to_numeric(getdata[1][col], 'coerce')

                getdata[1] = getdata[1].dropna()

                getdata[0].drop(getdata[0].head(1).index, inplace=True)
                getdata[0] = getdata[0].reset_index(drop=True)
                getdata[0].index = getdata[0].iloc[: , 0]
                getdata[0].drop(getdata[0].columns[0], axis=1, inplace=True)
                for (col1,col2) in zip(getdata[0].columns, getdata[1].columns):
                    getdata[0][col1] = pd.to_numeric(getdata[0][col1], 'coerce')
                    getdata[0].rename(columns={col1:col2}, inplace=True)

                getdata[1] = getdata[1].dropna()

                getdata[2].drop(getdata[2].head(1).index, inplace=True)
                getdata[2] = getdata[2].reset_index(drop=True)
                getdata[2].index = getdata[2].iloc[: , 0]
                getdata[2].drop(getdata[2].columns[0], axis=1, inplace=True)
                for col in getdata[2].columns:
                    getdata[2][col] = pd.to_numeric(getdata[2][col], 'coerce')

                getdata[2] = getdata[2].dropna()

                getdata[0] = getdata[0].T
                getdata[1] = getdata[1].T
                getdata[2] = getdata[2].T

                getdata[1] = getdata[1][['繼續營業單位稅前淨利（淨損）','基本每股盈餘合計']]
                getdata[1]['本期現金及約當現金增加（減少）數'] = getdata[2][['本期現金及約當現金增加（減少）數']]

                getdata[1] = getdata[1].T

                df_compsheet['%d'%(y+1)] = getdata[1][getdata[1].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')
                
        else:
            url = 'https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01Form&step=0&comp_id='+stock+'&YEAR1=%d&SEASON1=4&R_TYPE1=A' %(y)
            print (url)

            try:
                getdata=pd.read_html(url,encoding='utf16',header=0)
                getdata[1] = getdata[1].dropna()
                getdata[1].index = getdata[1].iloc[: , 0]
                getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
                for col in getdata[1].columns:
                    getdata[1][col] = pd.to_numeric(getdata[1][col], 'coerce')

                getdata[1].index.name = ''
                getdata[1] = getdata[1].T

                getdata[1] = getdata[1][['繼續營業單位稅前淨利(淨損)','基本每股盈餘淨額','本期現金及約當現金淨增減數']]

                getdata[1] = getdata[1].loc[:,~getdata[1].columns.duplicated()]
                
                col_names = ['繼續營業單位稅前淨利（淨損）','基本每股盈餘合計','本期現金及約當現金增加（減少）數']
                
                for iname in range(len(col_names)):
                    getdata[1].rename(columns={getdata[1].columns[iname]: col_names[iname]}, inplace=True)

                getdata[1] = getdata[1].T

                df_compsheet['%d'%(y+1)] = getdata[1][getdata[1].columns[0]]

            except Exception as e:
                print('get 404, please check if anything is wrong')

        #要睡覺一下，不然會被ben掉
        time.sleep(5)
        
    ### in case there is anything wrong
    ### skip the company -> check later
    try:
        df_compsheet.index.name = ''
        df_compsheet = df_compsheet.T
        df_compsheet.index = pd.to_datetime(df_compsheet.index)
        for col in df_compsheet.columns:
            df_compsheet[col] = pd.to_numeric(df_compsheet[col], 'coerce')

        df_compsheet = df_compsheet.sort_index()
        df_compsheet['淨利成長率'] = df_compsheet['繼續營業單位稅前淨利（淨損）'].pct_change()
        
    except Exception as e:
        print('Errors have occured! Check company ' + stock)
        
    df_compsheet.to_csv('./Financial_statement/'+stock+'.csv', encoding='utf-8')

    return df_compsheet

In [197]:
df_compsheet = financial_statement_bank_single('2867')
df_compsheet

2019
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=2867&SYEAR=2019&SSEASON=4&REPORT_ID=A
get 404, please check if anything is wrong
2018
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=2867&SYEAR=2018&SSEASON=4&REPORT_ID=A
2017
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=2867&SYEAR=2017&SSEASON=4&REPORT_ID=A
2016
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=2867&SYEAR=2016&SSEASON=4&REPORT_ID=A
2015
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=2867&SYEAR=2015&SSEASON=4&REPORT_ID=A
2014
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=2867&SYEAR=2014&SSEASON=4&REPORT_ID=A
get 404, please check if anything is wrong
2013
https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID=2867&SYEAR=2013&SSEASON=4&REPORT_ID=A
get 404, please ch

,繼續營業單位稅前淨利（淨損）,基本每股盈餘合計,本期現金及約當現金增加（減少）數,淨利成長率
2016-01-01,3529489.0,1.88,12616940.0,NaN
2017-01-01,2893025.0,1.78,-19750106.0,-0.180328
2018-01-01,3341161.0,1.71,3906756.0,0.154902
2019-01-01,-1533678.0,-0.14,6954080.0,-1.459025


---
## Test block ##

In [ ]:
df_compsheet = financial_statement('2812')


(df_compsheet.style
     .background_gradient('Reds',subset='營業收入合計')
     .background_gradient('Reds',subset='營業毛利（毛損）')
     .background_gradient('coolwarm',subset='營業利益（損失）')
     .background_gradient('coolwarm',subset='繼續營業單位稅前淨利（淨損）')
     .background_gradient('coolwarm',subset='基本每股盈餘合計')
     .background_gradient('coolwarm',subset='本期現金及約當現金增加（減少）數')
     .background_gradient('coolwarm',subset='營業毛利率(%)')
     .background_gradient('coolwarm',subset='營業利益率(%)')
     .background_gradient('coolwarm',subset='純益率(%)')
     .background_gradient('coolwarm',subset='股東權益率(%)')
     .background_gradient('coolwarm',subset='總資產報酬率(%)')
     .background_gradient('coolwarm',subset='營收成長率')
     .background_gradient('coolwarm',subset='淨利成長率')
     # .background_gradient('Greens', subset='Google search: Stock')
     .highlight_null()
)

In [182]:
stock = '2812'
y=2018
url = 'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID='+stock+'&SYEAR=%d&SSEASON=4&REPORT_ID=C' %(y)
# url = 'https://mops.twse.com.tw/server-java/t164sb01?t203sb01Form=t203sb01Form&step=1&CO_ID='+stock+'&SYEAR=%d&SSEASON=4&REPORT_ID=A' %(y)
print (url)
        
getdata=pd.read_html(url,encoding='utf16',header=0)

getdata[2].drop(getdata[2].head(1).index, inplace=True)
getdata[2] = getdata[2].reset_index(drop=True)
getdata[2].index = getdata[2].iloc[: , 0]
getdata[2].drop(getdata[2].columns[0], axis=1, inplace=True)
for col in getdata[2].columns:
    getdata[2][col] = pd.to_numeric(getdata[2][col], 'coerce')

getdata[2] = getdata[2].dropna()

getdata[1].drop(getdata[1].head(1).index, inplace=True)
getdata[1] = getdata[1].reset_index(drop=True)
getdata[1].index = getdata[1].iloc[: , 0]
getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
for (col1,col2) in zip(getdata[1].columns, getdata[2].columns):
    getdata[1][col1] = pd.to_numeric(getdata[1][col1], 'coerce')
    getdata[1].rename(columns={col1:col2}, inplace=True)

getdata[1] = getdata[1].dropna()

getdata[3].drop(getdata[3].head(1).index, inplace=True)
getdata[3] = getdata[3].reset_index(drop=True)
getdata[3].index = getdata[3].iloc[: , 0]
getdata[3].drop(getdata[3].columns[0], axis=1, inplace=True)
for col in getdata[3].columns:
    getdata[3][col] = pd.to_numeric(getdata[3][col], 'coerce')

getdata[3] = getdata[3].dropna()

getdata[1] = getdata[1].T
getdata[2] = getdata[2].T
getdata[3] = getdata[3].T

getdata[2] = getdata[2][['繼續營業單位稅前淨利（淨損）','營業費用合計','基本每股盈餘合計']]
getdata[2]['本期現金及約當現金增加（減少）數'] = getdata[3][['本期現金及約當現金增加（減少）數']]

# if y==2017:
#     getdata[1]['股東權益總額'] = getdata[0]['權益總計']
# else:
#     getdata[1]['股東權益總額'] = getdata[0]['權益總額']

# if y==2014 or y==2013:
#     getdata[1]['資產總額'] = getdata[0]['資產總額']
# else:
#     getdata[1]['資產總額'] = getdata[0]['資產總計']

# getdata[2] = getdata[2].T

getdata[2]


https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=2812&SYEAR=2018&SSEASON=4&REPORT_ID=C


會計項目,繼續營業單位稅前淨利（淨損）,營業費用合計,基本每股盈餘合計,本期現金及約當現金增加（減少）數
2018年度,4759883.0,6456769.0,1.18,-214999.0
2017年度,4355212.0,5914684.0,1.08,-48153382.0


In [149]:
stock = '2493'
y=2012
url = 'https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01Form&step=0&comp_id='+stock+'&YEAR1=%d&SEASON1=4&R_TYPE1=A' %(y)
print (url)

getdata=pd.read_html(url,encoding='utf16',header=0)
getdata[1] = getdata[1].dropna()
getdata[1].index = getdata[1].iloc[: , 0]
getdata[1].drop(getdata[1].columns[0], axis=1, inplace=True)
for col in getdata[1].columns:
    getdata[1][col] = pd.to_numeric(getdata[1][col], 'coerce')

getdata[1].index.name = ''
getdata[1] = getdata[1].T

getdata[1] = getdata[1][['營業收入合計','營業毛利(毛損)','營業淨利(淨損)',
                            '繼續營業單位稅前淨利(淨損)','營業費用合計','本期淨利(淨損)','基本每股盈餘淨額',
                            '本期現金及約當現金淨增減數','股東權益總額','資產總額']]

col_names = ['營業收入合計','營業毛利（毛損）','營業利益（損失）',
            '繼續營業單位稅前淨利（淨損）','營業費用合計','本期淨利（淨損）','基本每股盈餘合計','本期現金及約當現金增加（減少）數','股東權益總額','資產總額']

getdata[1] = getdata[1].loc[:,~getdata[1].columns.duplicated()]

for iname in range(len(col_names)):
    print (iname,getdata[1].columns[iname],col_names[iname])
    getdata[1].rename(columns={getdata[1].columns[iname]: col_names[iname]}, inplace=True)

getdata[1] = getdata[1].T

getdata[1]

https://mops.twse.com.tw/server-java/t147sb02?t203sb01Form=t203sb01Form&step=0&comp_id=2493&YEAR1=2012&SEASON1=4&R_TYPE1=A
0 營業收入合計 營業收入合計
1 營業毛利(毛損) 營業毛利（毛損）
2 營業淨利(淨損) 營業利益（損失）
3 繼續營業單位稅前淨利(淨損) 繼續營業單位稅前淨利（淨損）
4 營業費用合計 營業費用合計
5 本期淨利(淨損) 本期淨利（淨損）
6 基本每股盈餘淨額 基本每股盈餘合計
7 本期現金及約當現金淨增減數 本期現金及約當現金增加（減少）數
8 股東權益總額 股東權益總額
9 資產總額 資產總額


,本期金額,去年同期金額
,,
營業收入合計,2294911.00,2404941.00
營業毛利（毛損）,615125.00,731762.00
營業利益（損失）,313353.00,425441.00
繼續營業單位稅前淨利（淨損）,351355.00,492142.00
營業費用合計,301772.00,306321.00
本期淨利（淨損）,346762.00,396817.00
基本每股盈餘合計,3.03,3.47
本期現金及約當現金增加（減少）數,-207449.00,274544.00
股東權益總額,2145232.00,2174413.00


---
## Obsolete ##

In [3]:
#----------------- （１）評估價值是否被低估？（股票價格不會太貴） -------------
########## 去公開資訊觀測站，把本益比、股價淨值比爬下來 ##########
url = 'http://www.twse.com.tw/exchangeReport/BWIBBU_d?response=json&date=&selectType=&_=' + str(time.time())
print (url)
list_req = requests.get(url)
soup = BeautifulSoup(list_req.content, "html.parser")
getjson=json.loads(soup.text)

# 因為是表格式，用dataframe處理會比較方便
stockdf = pd.DataFrame(getjson['data'],columns=["證券代號","證券名稱","殖利率(%)","股利年度","本益比","股價淨值比","財報年/季"])
PBR = pd.to_numeric(stockdf['股價淨值比'], errors='coerce') < 0.5 # 找到股價淨值比小於0.7的股票
EPS = pd.to_numeric(stockdf['本益比'], errors='coerce') < 15 # 找到本益比小於15的股票
candidate= stockdf[(PBR & EPS)] # 綜合以上兩者，選出兩者皆符合的股票
candidate

http://www.twse.com.tw/exchangeReport/BWIBBU_d?response=json&date=&selectType=&_=1576944476.0300171


,證券代號,證券名稱,殖利率(%),股利年度,本益比,股價淨值比,財報年/季
431,2611,志信,5.86,107,14.29,0.48,108/3
468,2841,台開,0.00,107,14.77,0.35,108/3
883,8429,金麗-KY,2.67,107,10.23,0.30,108/3
